# Self borrowing activity on Compound V2

This notebook intends to illustrate the correlation between self-borrowing and the incentives intern to the protocol. 

In [53]:
import altair as alt
import pandas as pd
import numpy as np


liability = pd.read_csv('../data/daily_liability_matrix.csv') # Load market price data
liability

,symbol_bor,symbol_col,account,matchBorrowUSD,matchCollateralUSD,effectiveUSD,date
0,cBAT,cBAT,1,1.028105e+01,1.763471e+02,1.028105e+01,2019-05-07
1,cBAT,cETH,2,1.895208e+01,3.152674e+02,1.895208e+01,2019-05-07
2,cBAT,cREP,1,2.004187e+00,3.437709e+01,2.004187e+00,2019-05-07
3,cBAT,cSAI,1,2.686493e+01,4.608043e+02,2.686493e+01,2019-05-07
4,cBAT,cUSDC,1,8.139770e+00,1.396185e+02,8.139770e+00,2019-05-07
...,...,...,...,...,...,...,...
209063,cZRX,cUSDC,48,1.068964e+05,1.503970e+06,1.068959e+05,2023-07-31
209064,cZRX,cWBTC,15,1.554430e+02,4.561121e+02,1.547590e+02,2023-07-31
209065,cZRX,cWBTC2,12,1.970710e+02,3.269187e+02,1.953021e+02,2023-07-31
209066,cZRX,cYFI,1,8.481281e-10,1.729400e-05,8.481281e-10,2023-07-31


In [54]:
self_borrow = liability[liability['symbol_bor'] == liability['symbol_col']].reset_index().drop('symbol_col', axis=1)
self_borrow

,index,symbol_bor,account,matchBorrowUSD,matchCollateralUSD,effectiveUSD,date
0,0,cBAT,1,1.028105e+01,1.763471e+02,1.028105e+01,2019-05-07
1,7,cETH,1,4.655172e+01,7.984846e+02,4.655172e+01,2019-05-07
2,14,cREP,1,5.968725e-01,1.023794e+01,5.968725e-01,2019-05-07
3,21,cSAI,1,8.015008e+01,1.374785e+03,8.015008e+01,2019-05-07
4,28,cUSDC,1,1.211813e+01,2.078579e+02,1.211813e+01,2019-05-07
...,...,...,...,...,...,...,...
14872,208989,cUSDC,424,3.926036e+07,8.587211e+07,3.926026e+07,2023-07-31
14873,209028,cWBTC,26,6.120590e+02,1.448418e+03,6.101344e+02,2023-07-31
14874,209041,cWBTC2,23,6.802807e+05,1.062553e+06,6.802807e+05,2023-07-31
14875,209054,cYFI,2,3.216983e+00,5.745345e+00,3.216983e+00,2023-07-31


In [55]:

daily_pools = self_borrow.groupby(['symbol_bor', 'date']).sum().reset_index()
daily_pools

,symbol_bor,date,index,account,matchBorrowUSD,matchCollateralUSD,effectiveUSD
0,cAAVE,2021-08-19,60189,1,2489.868402,6224.671004,2489.868402
1,cAAVE,2021-08-20,60356,1,2722.808655,6805.843010,2722.808655
2,cAAVE,2021-08-21,60524,1,4610.444347,11525.155716,4610.444347
3,cAAVE,2021-08-22,60697,1,4503.303663,11255.177213,4503.303663
4,cAAVE,2021-08-23,60870,1,4627.488659,11563.982738,4627.488659
...,...,...,...,...,...,...,...
14872,cZRX,2023-07-27,208167,28,52935.397436,641694.453112,52935.397431
14873,cZRX,2023-07-28,208392,28,52933.375206,641689.753783,52933.375201
14874,cZRX,2023-07-29,208617,28,52931.189305,641687.806921,52931.189300
14875,cZRX,2023-07-30,208842,28,52933.307208,641690.207267,52933.307203


In [56]:
graph = alt.Chart(daily_pools[daily_pools['symbol_bor'].isin(['cUSDC', 'cCOMP', 'cDAI', 'cETH', 'cZRX', 'cWBTC2'])]).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x='date:T',
    y='matchBorrowUSD',
    color='symbol_bor'
)

graph

alt.Chart(...)

Little historical view regarding the rewards:

- **June 15th, 2020**: Every Ethereum block, 0.50 COMP will be distributed across ETH, DAI, USDC, USDT, BAT, REP, WBTC and ZRX markets

- **June 27th, 2020**: Lower the number of distributed COMP to 0.44

- **August 31st, 2020**: Reduce COMP emissions by 20%

- **October 17th, 2020**: Add COMP Support

- **September 29th, 2021**: two different COMP distribution rates for each and every market - borrow-side rate and supply-side rate.

- **November 12th, 2021**: End cCOMP Borrow Rewards

- **March 27th, 2022**: the governance has voted to cut the existing rewards by 50%

- **June 29th, 2023**: Top-up COMP Rewards

Let's check the number of users over time.

In [84]:

market = pd.read_csv('../data/daily_markets.csv') # Load market price data
market

,borrowRate,cash,collateralFactor,exchangeRate,interestRateModelAddress,name,reserves,supplyRate,symbol,cTokenAddress,...,underlyingPrice,underlyingSymbol,accrualBlockNumber,blockTimestamp,borrowIndex,reserveFactor,underlyingPriceUSD,underlyingDecimals,date,newPrice
0,0.039471,1.440748e+01,0.50,0.020000,0xbae04cbf96391086dc643e842b517734e214d698,Compound Augur,0.000000e+00,0.002306,cREP,0x158079ee67fce2f58472a96584a73c7ab9ac95c1,...,0.116547,REP,7715069,1557250486,1.000041,100000000000000000,20.004143,18,2019-05-07,NaN
1,0.016183,1.135850e+03,0.75,0.020000,0xc64c4cba055efa614ce01f4bad8a9f519c4f8fab,Compound USD Coin,6.000000e-06,0.001179,cUSDC,0x39aa39c021dfbae8fac545936693ac917d5e7563,...,0.005846,USDC,7716064,1557264374,1.000001,100000000000000000,1.000000,6,2019-05-07,NaN
2,0.012274,1.530022e+01,0.75,0.020000,0xc64c4cba055efa614ce01f4bad8a9f519c4f8fab,Compound Ether,4.704680e-06,0.000678,cETH,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,...,1.000000,ETH,7716242,1557266749,1.000047,100000000000000000,171.043415,18,2019-05-07,NaN
3,0.033222,4.381321e+03,0.60,0.020000,0xbae04cbf96391086dc643e842b517734e214d698,Compound Basic Attention Token,4.561337e-04,0.001318,cBAT,0x6c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e,...,0.001898,BAT,7712572,1557217208,1.000026,100000000000000000,0.335843,18,2019-05-07,NaN
4,0.054585,7.674348e+02,0.60,0.020000,0xbae04cbf96391086dc643e842b517734e214d698,Compound 0x,1.724746e-04,0.005664,cZRX,0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407,...,0.001559,ZRX,7711781,1557206685,1.000021,100000000000000000,0.276315,18,2019-05-07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21108,0.021737,1.359424e+04,0.70,0.020076,0xf2e5db36b0682f2cd6bc805c3a4236194e01f4d5,Compound Wrapped BTC,1.206402e+02,0.000134,cWBTC2,0xccf4429db6322d5c611ee964527d42e5d685dd6a,...,15.738445,WBTC,17816367,1690847207,1.086955,200000000000000000,29204.203950,8,2023-07-31,NaN
21109,0.026027,2.294171e+04,0.73,0.020626,0xd956188795ca6f4a74092ddca33e0ea4ca3a1395,Compound Aave Token,5.945144e+02,0.000523,cAAVE,0xe65cdb6479bac1e22340e4e755fae7e509ecd06c,...,0.035477,AAVE,17814861,1690829063,1.161843,250000000000000000,66.062500,18,2023-07-31,NaN
21110,0.050178,1.688357e+05,0.00,0.021450,0xa1046abfc2598f48c44fb320d281d3f3c0733c9a,Compound Dai,9.199748e+00,0.000000,cSAI,0xf5dce57282a584d2746faf1593d3121fcac444dc,...,0.005285,DAI,17691182,1690125815,1.388610,1000000000000000000,9.904040,18,2023-07-31,NaN
21111,0.037653,5.667166e+07,0.00,0.022459,0xfb564da37b41b2f6b6edcc3e56fbf523bd9f2012,Compound USDT,3.608744e+06,0.026228,cUSDT,0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9,...,0.000538,USDT,17816239,1690845659,1.203922,75000000000000000,1.000000,6,2023-07-31,NaN


In [86]:
users = market['underlyingAddress'].value_counts().reset_index()
users.columns = ['underlyingAddress', 'n_users']

users


,underlyingAddress,n_users
0,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,2342
1,0x1985365e9f78359a9b6ad760e32412f4a445e862,1547
2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,1547
3,0x0000000000000000000000000000000000000000,1547
4,0x0d8775f648430679a709e98d2b0cb6250d2887ef,1547
5,0xe41d2489571d322189246dafa5ebde1f4699f498,1547
6,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,1547
7,0x6b175474e89094c44da98b954eedeac495271d0f,1342
8,0xdac17f958d2ee523a2206206994597c13d831ec7,1187
9,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,1032


#To be explained